<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
 

<strong>Objectives:</strong>

    - Estudiar y explorar las series correspondientes al producto 68
    - Comprobar las estacionalidades
    - Descomponer la serie
    - Obtener la media por día de la semana, mes, etc.
    - Estudiar la asignación de missings en ventas y stock
    
    
<strong>Conclusions:</strong>

   
</div>

In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.load_data import load_csv, load_data
from src.prepare_data import prepare_train_data

In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_seasonal_descompose(serie, period):
    
    result = seasonal_decompose(serie, model='additive', period=period)
    
    plt.figure(figsize=(20,40))
    plt.subplot(4,1,1)
    result.observed.plot(title='Observed')
    plt.subplot(4,1,2)
    result.trend.plot(title='Trend')
    plt.subplot(4,1,3)
    result.seasonal.plot(title='Seasonal - Period {}'.format(period))
    plt.subplot(4,1,4)
    result.resid.plot(title='Residual')
    plt.tight_layout()

In [3]:
df = load_data()
df.head()

===================  LOAD DATASET stock ====================
Reading CSV in data/raw/03_TablaStock.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 628.
============  CLEAN STOCK  =============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). Rows dropped: 1784
[WARNING] Number of dates missings:                                83
[WARNING] Number of dates missings (droping sundays and festivos): 63
[WARNING] Stock data with different units for same product & data. Rows: 12666
Dataset stock loaded. Shape: (25518, 3)

===================  LOAD DATASET ventas ===================
Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN VENTAS  ============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). 

/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['missing'] = (ts.udsstock.isna()) * 1
/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data["roll4wd_" + col] = day_data.loc[(day_data.festivo == 0), col].rolling(4, win_type='triang', min_periods=1).mean()
/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_va

Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and 

Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for

Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for

...

Get stock missing type
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by 

Get stock missing type
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by 

Get stock missing type
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by 

Get stock missing type
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by 

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,weekday,quarter,month,...,working_day,sin_weekday,cos_weekday,stockMissingType,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa
0,2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,2833.0,3405.131579,18275.0,11012.218750,102304.0,37273.000000
1,2019-06-05,10,2267.0,1874.0,36888.0,1.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,1874.0,2440.763158,2267.0,1822.161290,36888.0,10102.540541
2,2019-06-05,11,2506.0,2755.0,31078.0,0.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,2755.0,2323.135135,2506.0,3987.666667,31078.0,10973.815789
3,2019-06-05,12,1279.0,1161.0,33661.0,0.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,1161.0,1143.368421,1279.0,1605.741935,33661.0,5724.236842
4,2019-06-05,13,2493.0,1603.0,32119.0,1.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,1603.0,1874.447368,2493.0,3113.843750,32119.0,8500.421053


In [4]:
df.loc[df.promo.isna()]

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,weekday,quarter,month,...,working_day,sin_weekday,cos_weekday,stockMissingType,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa


In [5]:
data10=prepare_train_data(df.loc[df.producto == "10"])

data10.info()

===================  FILTER TRAIN DATA  ====================
Dropped rows corresponding to 23 to 26-03-2020 for not having the ventas data for these days.
Rows dropped: 4
Assigned missings in udsventa.
Assigned missings in udsprevisionempresa filling with 0
Assigning missings for udsstock
Missing in dataset:               88 (292 total rows).
Missings in holiday days:         20 (50 total rows).
Missings in working days:         68 (242 total rows).
Assigned missings for working days data with the 4last wd rolling window - Remaining missings: 26
Assigned left missings for working days data with the mean of the weekday - Remaining missings: 20
Assigned missings for holiday data and no active business data - Remaining missings:      0
Get shifted variable for udsprevisionempresa with period 1
Get shifted variable for udsprevisionempresa with period 2
Get shifted variable for udsprevisionempresa with period 3
Get shifted variable for udsprevisionempresa with period 4
Get shifted variable 

In [6]:
data10.loc[data10.weekday.isna()]

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,weekday,quarter,month,...,udsprevisionempresa_shifted1,udsprevisionempresa_shifted2,udsprevisionempresa_shifted3,udsprevisionempresa_shifted4,udsprevisionempresa_shifted5,udsprevisionempresa_shifted6,udsprevisionempresa_shifted7,udsstock_diff-1,udsstock_diff1,udsventa_diff1


In [11]:
def _get_roll4wd(day_data, col):
    day_data["roll4wd_" + col] = day_data[col].rolling(4, win_type='triang', min_periods=1).mean()
    day_data["meanwd_" + col] = day_data[col].mean()
    print("Getting rolling windows of last 5 days by product and weekday for column {}".format(col))
    return day_data

def get_dateproduct_variables(prod_data):
#     drop_cols = prod_data.loc[:,~prod_data.columns.isin(["fecha", "producto"])].columns
#     out_prod_data = prod_data.copy()
    out = pd.DataFrame({})
    for wd in prod_data['weekday'].unique():
        print(wd)
        day_data = prod_data.loc[(prod_data.festivo == 0) & (prod_data.weekday == wd)]
        # Get roll4wd and meanwd for stock, venta and prevision
        for col in ['udsventa', 'udsstock', 'udsprevisionempresa']:
            day_data = _get_roll4wd(day_data, col)
        out = pd.concat([out, day_data])
#     out_prod_data = prod_data.drop(drop_cols, axis=1)
#     out_prod_data = out_prod_data.merge(out, how='left', on=['fecha','producto'])
    return out

data10 = df.loc[df.producto == "10"]

get_dateproduct_variables(data10)

2
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
3
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
4
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa
5
Getting rolling windows of last 5 days by product and weekday for column udsventa
Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and wee

<ipython-input-11-7b592bed55bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data["roll4wd_" + col] = day_data[col].rolling(4, win_type='triang', min_periods=1).mean()
<ipython-input-11-7b592bed55bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data["meanwd_" + col] = day_data[col].mean()


,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,weekday,quarter,month,...,working_day,sin_weekday,cos_weekday,stockMissingType,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa
1,2019-06-05,10,2267.0,1874.0,36888.0,1.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,1874.000000,2440.763158,2267.000000,1822.16129,36888.000000,10102.540541
505,2019-06-12,10,1860.0,2184.0,6916.0,0.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,1951.500000,2440.763158,2165.250000,1822.16129,29395.000000,10102.540541
1009,2019-06-19,10,1589.0,3394.0,20616.0,1.0,0.0,2,2,6,...,True,0.974928,-0.222521,0.0,2224.000000,2440.763158,1995.714286,1822.16129,21718.285714,10102.540541
1513,2019-06-26,10,NaN,1726.0,13744.0,1.0,0.0,2,2,6,...,True,0.974928,-0.222521,2.0,2541.750000,2440.763158,1802.000000,1822.16129,16653.500000,10102.540541
2017,2019-07-03,10,1802.0,900.0,NaN,1.0,0.0,2,3,7,...,True,0.974928,-0.222521,0.0,2305.500000,2440.763158,1685.800000,1822.16129,15713.714286,10102.540541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19081,2020-02-25,10,1473.0,870.0,4649.0,0.0,0.0,1,1,2,...,True,0.781831,0.623490,0.0,1433.000000,1824.051282,2088.000000,1601.15625,9798.375000,7792.179487
19585,2020-03-03,10,830.0,1889.0,11406.0,0.0,0.0,1,1,3,...,True,0.781831,0.623490,0.0,1263.250000,1824.051282,1789.750000,1601.15625,8850.125000,7792.179487
20089,2020-03-10,10,426.0,1682.0,7335.0,0.0,0.0,1,1,3,...,True,0.781831,0.623490,0.0,1410.875000,1824.051282,1217.250000,1601.15625,8389.125000,7792.179487
20593,2020-03-17,10,1259.0,3926.0,4436.0,1.0,0.0,1,1,3,...,True,0.781831,0.623490,0.0,1938.625000,1824.051282,812.500000,1601.15625,8163.500000,7792.179487


In [44]:
from src.create_variables import _get_shifted, _get_diff,_get_stockMissingTypeByProd
def get_product_variables(data):
    # Columnas a eliminar antes de hacer el merge
    drop_cols = data.loc[:,~data.columns.isin(["fecha", "producto"])].columns
    out_data = data.copy()
    # Creamos un dataframe vacio
    out = pd.DataFrame({})
    for product in ["1"]: #data['producto'].unique():
        prod_data = data.loc[(data['producto'] == product)]
        # Creamos las variables de shifted prevision
#         for period in range(1,8):
#             prod_data = _get_shifted(prod_data, 'udsprevisionempresa', period)
#         # Creamos las variables diff +1 y diff -1 para stock
#         for period in [-1,1]:
#             prod_data = _get_diff(prod_data, 'udsstock', period)
#         # Creamos la variabla diff +1 para ventas
        prod_data = _get_diff(prod_data, 'udsventa', period = 1)
        # Creamos la variable de tipo stock missing
        prod_data = _get_stockMissingTypeByProd(prod_data)
        # Get variables of date prod
        # prod_data = get_dateproduct_variables(prod_data)
        print(prod_data.head())
        
        out = pd.concat([out, prod_data])
    out_data = data.drop(drop_cols, axis=1)
    out_data = out_data.merge(prod_data, how='left', on=['fecha','producto'])
    return out_data

test = get_product_variables(test)

Get diff variable for udsventa with period 1
Get stock missing type
       fecha producto  udsstock  udsventa  udsprevisionempresa  promo  \
0 2019-06-05        1   18275.0    2833.0             102304.0    0.0   
1 2019-06-06        1       NaN   17475.0              34479.0    1.0   
2 2019-06-07        1       NaN    4250.0              25127.0    0.0   
3 2019-06-08        1       NaN       NaN              98716.0    0.0   
4 2019-06-09        1       NaN       NaN                  NaN    0.0   

   festivo  udsventa_diff1  stockMissingType  
0      0.0             NaN               0.0  
1      0.0         14642.0               2.0  
2      0.0        -13225.0               2.0  
3      0.0             NaN               2.0  
4      0.0             NaN               2.0  


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_data[col_name] = prod_data[column].diff(periods=period)
/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['missing'] = (ts.udsstock.isna()) * 1


In [45]:
test.loc[test.producto == "1"]

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,udsventa_diff1,stockMissingType
0,2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0,NaN,0.0
72,2019-06-06,1,NaN,17475.0,34479.0,1.0,0.0,14642.0,2.0
144,2019-06-07,1,NaN,4250.0,25127.0,0.0,0.0,-13225.0,2.0
216,2019-06-08,1,NaN,NaN,98716.0,0.0,0.0,NaN,2.0
288,2019-06-09,1,NaN,NaN,NaN,0.0,0.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...
20952,2020-03-22,1,4283.0,NaN,NaN,0.0,0.0,NaN,0.0
21024,2020-03-23,1,4283.0,NaN,2267.0,0.0,0.0,NaN,0.0
21096,2020-03-24,1,4344.0,NaN,3211.0,0.0,0.0,NaN,0.0
21168,2020-03-25,1,1058.0,NaN,13036.0,0.0,0.0,NaN,0.0


In [28]:
df

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo
0,2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0
1,2019-06-05,10,2267.0,1874.0,36888.0,1.0,0.0
2,2019-06-05,11,2506.0,2755.0,31078.0,0.0,0.0
3,2019-06-05,12,1279.0,1161.0,33661.0,0.0,0.0
4,2019-06-05,13,2493.0,1603.0,32119.0,1.0,0.0
...,...,...,...,...,...,...,...
21307,2020-03-26,95,NaN,NaN,NaN,0.0,0.0
21308,2020-03-26,96,465.0,NaN,266.0,1.0,0.0
21309,2020-03-26,97,395.0,NaN,235.0,0.0,0.0
21310,2020-03-26,98,95.0,NaN,243.0,0.0,0.0


In [11]:
data.loc[:,~data.columns.isin(["fecha", "producto"])].columns

Index(['udsstock', 'udsventa', 'udsprevisionempresa', 'promo', 'festivo',
       'stockMissingType', 'weekday', 'quarter', 'month', 'weekofyear',
       'working_day', 'sin_weekday', 'cos_weekday', 'deltaStock',
       'roll4wd_udsventa', 'meanwd_udsventa', 'roll4wd_udsstock',
       'meanwd_udsstock', 'roll4wd_udsprevisionempresa',
       'meanwd_udsprevisionempresa', 'udsprevision_1', 'udsprevision_2',
       'udsprevision_3', 'udsprevision_4', 'udsprevision_5', 'udsprevision_6',
       'udsprevision_7'],
      dtype='object')

In [10]:
data

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,...,weekofyear,deltaStock,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa,holiday_udsstock,udsprevision1
0,2019-06-05,68,1545.000000,392.0,7451.0,0.0,0.0,0.0,2,2,...,23,NaN,392.000,428.589744,1545.0,1007.093750,7451.00,1753.947368,1545.0,0.0
1,2019-06-06,68,964.343750,428.0,9499.0,0.0,0.0,2.0,3,2,...,23,NaN,428.000,430.425000,NaN,964.343750,9499.00,2238.692308,1429.0,7451.0
2,2019-06-07,68,794.533333,510.0,5106.0,0.0,0.0,2.0,4,2,...,23,NaN,510.000,506.641026,NaN,794.533333,5106.00,1444.564103,1429.0,9499.0
3,2019-06-08,68,1140.357143,81.0,1735.0,0.0,0.0,2.0,5,2,...,23,NaN,81.000,105.850000,NaN,1140.357143,1735.00,715.736842,1429.0,5106.0
4,2019-06-09,68,1429.000000,0.0,0.0,0.0,0.0,2.0,6,2,...,23,NaN,NaN,32.500000,NaN,918.208333,NaN,NaN,1429.0,1735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,2020-03-18,68,1019.000000,587.0,2303.0,0.0,0.0,0.0,2,1,...,12,291.0,584.875,428.589744,1314.4,1007.093750,2518.50,1753.947368,1019.0,891.0
288,2020-03-19,68,728.000000,654.0,1926.0,0.0,0.0,0.0,3,1,...,12,-363.0,517.750,430.425000,899.0,964.343750,2910.75,2238.692308,728.0,2303.0
289,2020-03-20,68,1091.000000,678.0,1190.0,0.0,0.0,0.0,4,1,...,12,-1304.0,773.000,506.641026,956.4,794.533333,2168.50,1444.564103,1091.0,1926.0
290,2020-03-21,68,2395.000000,66.0,348.0,0.0,0.0,0.0,5,1,...,12,87.0,219.875,105.850000,818.0,1140.357143,933.25,715.736842,2395.0,1190.0


In [ ]:
data = data[["udsventa","udsprevisionempresa", "udsstock"]]

data.info()

In [ ]:
actual = np.array([1,2,3,1])
err_naive = np.mean(np.diff(np.array(actual)))

In [ ]:
actual = np.array([1,2,3,1])
np.mean(np.abs(np.diff(np.array(actual))))

# Time Series

In [ ]:
subdf = df.loc[(df.producto=='68')]
for series in ['udsventa', 'udsstock', 'udsprevisionempresa']:
    subdf[[series, 'roll4wd_'+series]].plot(figsize=(20,10), title="Producto: {}".format('68'))
    plt.show()
    


In [ ]:
def _assing_missings_stock(df, col):
    """Assign missings: backfill for holiday
                        4wd rolling windows for else???
    """
    print("Assigning missings for {}".format(col))
    data = df.copy()
    holiday_data = data.loc[(data['festivo'] == 1) | (data['weekday'] == 6)]
    wd_data = data.loc[(data['festivo'] != 1) & (data['weekday'] != 6)]
    
    print("Missing in dataset:               {} ({} total rows).".format(sum(data[col].isna()), data.shape[0]))
    print("Missings in holiday days:         {} ({} total rows).".format(sum(holiday_data[col].isna()), holiday_data.shape[0]))
    print("Missings in working days:         {} ({} total rows).".format(sum(wd_data[col].isna()), wd_data.shape[0]))
    
    data['holiday_' + col] = data.fillna(method='bfill')
    print("Assigned missings for holiday data - Remaining missings:      {}".format(sum(data[col].isna())))
    
    data.loc[(data['festivo'] != 1) & (data['weekday'] != 6), col] = wd_data[col].fillna(wd_data["roll4wd_" + col])
    print("Assigned missings for working days data - Remaining missings: {}".format(sum(data[col].isna())))
    return data
col = 'udsstock'
data = subdf.copy()

# Creamos vectores de filtro
holiday = (data['festivo'] == 1) | (data['weekday'] == 6)
workday = (data['festivo'] != 1) & (data['weekday'] != 6)
missing = data[col].isna()
incrementnot0 = (~data['deltaStock'].isna()) & (data['deltaStock'] != 0)


print("Missing in dataset:               {} ({} total rows).".format(sum(missing), data.shape[0]))
print("Missings in holiday days:         {} ({} total rows).".format(sum(missing & holiday), sum(holiday)))
print("Missings in working days:         {} ({} total rows).".format(sum(missing & workday), sum(workday)))

print("In holiday days, the increment of stock was different from 0 in {} days.".format(sum(holiday & incrementnot0)))

# Creamos una columna para los holiday missings
data['holiday_missing'] = data[col].fillna(method='bfill')

# Asignamos todos los valores como rolling windows
data[col] = data[col].fillna(data["roll4wd_" + col])
print("Assigned missings for working days data - Rolling windows method.")

# Asignamos los holidays por encima y borramos la columna intermedia
data.loc[holiday & missing, col] = data.loc[holiday & missing, "holiday_missing"]
data = data.drop("holiday_missing", axis=1)
print("Assigned missings for holiday data - backfill method")
print("Assigned missings for holiday data - Remaining missings: {}".format(sum(data[col].isna())))



data


In [ ]:
subdf = df.loc[(df.producto=='1')]
subdf['deltaStock1'] = subdf.udsstock.diff(periods=1)
subdf.loc[subdf.weekday == 6]

In [ ]:
plot_seasonal_descompose(subdf.udsventa, 7)

## Missings de ventas

In [ ]:
miss[['weekday','udsventa']].fillna(0).groupby("weekday")['udsventa'].count()

In [ ]:
data = subdf
col = 'roll5wd_udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))


# Asignamos 

In [ ]:
data = subdf
col = 'udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))

In [ ]:
no_miss.loc[no_miss.deltaStock == 0]

In [ ]:
miss.loc[miss.deltaStock == 0]

In [ ]:
subdf.loc[subdf.weekday == 6]

In [ ]:
subdf.loc[subdf.index > '2019-08-12']

## Medias de las ventas

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].mean()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].max()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].max()

In [ ]:
subdf

## Ventas en festivo o domingo faltantes puede ser 0?

In [ ]:
subdf = df.loc[(df.producto=='68')]
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[subdf.udsventa.isna()]